In [1]:
## Some imports
from __future__ import annotations
from dataclasses import dataclass

In [2]:
# Create a curve dataclass

@dataclass
class Curve:
    """
    Ellipitic curve over the field of integers modulo a prime.
    Points on the curve satisfy y^2 = x^3 + a*x + b (mod p).
    """
    p: int # the prime modulus of the finite field
    a: int
    b: int

# secp256k1 uses a = 0, b = 7 so the equation will be y^2 = x^3 + 7 (mod p)

bitcoin_curve = Curve(
    p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F,
    a = 0x0000000000000000000000000000000000000000000000000000000000000000, # a = 0
    b = 0x0000000000000000000000000000000000000000000000000000000000000007, # b = 7
)

## Creating a Generator point.

### A fixed starting point on the curve's cycle, which kicks off the random walk around the curve. This is a publicly known and agreed upon constant.

In [3]:
@dataclass
class Point:
    """ An integer point (x, y) on a Curve """
    curve: Curve
    x: int
    y: int
        
G = Point(
    bitcoin_curve,
    x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
    y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8,
)

# Confirm that the generator point is on the curve
print("Generator is on the curve", (G.y**2 - G.x**3 - 7) % bitcoin_curve.p == 0)

# Try a totally random point to check it will not be on the curve (most likely)

import random
random.seed(2009)
x = random.randrange(0, bitcoin_curve.p)
y = random.randrange(0, bitcoin_curve.p)
print("Random point is not on the curve: ", (y**2 - x**3 - 7) % bitcoin_curve.p == 0)

Generator is on the curve True
Random point is not on the curve:  False


### If the order of generating point G is known, and is the size of the set, lets put it in a structure called Generator

In [4]:
@dataclass
class Generator:
    """
    A generator over a curve: an initial point and the (pre-computed) order
    """
    G: Point     # a generator point on the curve
    n: int       # the order of the generating point, so 0*G = n*G = INF
        
bitcoin_gen = Generator(
    G = G,
    # the order of G is known and can be mathematically derived
    n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141,
)

### We have not done anything yet, only defined data structures, and filled them with publicly known constants related to elliptic curves used in Bitcoin.

### Now we will generate our private key. That is, a random integer that satisfies 1 <= key < n (n is the order of G)

In [5]:
# The post I am following uses a deterministic way of doing it, 
# for reproducability, I will use a real one
secret_key = random.randrange(1, bitcoin_gen.n) 
assert 1 <= secret_key < bitcoin_gen.n
print(secret_key)
# generated key: 104966969588467315726631053946449615147380658266902276382595870948465674913455

104966969588467315726631053946449615147380658266902276382595870948465674913455


In [6]:
### Now to generate a public key

INF = Point(None, None, None) # special point at "infinity", similar to zero

def extended_euclidian_algorithm(a, b):
    """
    Returns (gcd, x, y) s.t. a * x + b * y == gcd
    This function implements the extended Euclidian
    algorithm and runs in 0(log b) in the worst case,
    taken from Wikipedia.
    """
    
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient* t
    return old_r, old_s, old_t

def inv(n, p):
    """ Returns modular multipicative inverse m s t (n * m) % p == 1"""
    gcd, x, y = extended_euclidian_algorithm(n, p) # pylint: disable=unused-variable
    return x % p

def elliptic_curve_addition(self, other: Point) -> Point:
    # handle special case of P + 0 = 0 + P = 0
    if self == INF:
        return other
    if other == INF:
        return self
    # handle special case of P + (-P) = 0
    if self.x == other.x and self.y != other.y:
        return INF
    # compute the "slope"
    if self.x == other.x: # (self.y = other.y is guaranteed per above check)
        m = (3 * self.x**2 + self.curve.a) * inv(2 * self.y, self.curve.p)
    else:
        m = (self.y - other.y) * inv(self.x - other.x, self.curve.p)
    # compute the new point
    
    rx = (m**2 - self.x - other.x) % self.curve.p
    ry = (-(m*(rx - self.x) + self.y)) % self.curve.p
    return Point(self.curve, rx, ry)

Point.__add__ = elliptic_curve_addition # added to point class
    
    

### Note that the above may seem scary, but at the heart it is adding and multiplying of tuples (x, y) with some modulo p sprinkled in

### Lets try it out by generating some keypairs

In [7]:
# if our secret was the integer 1, then our public key would just be G
sk = 1
pk = G
print(f" secret key: {sk}\n public key: {(pk.x, pk.y)}")
print("Verify the public key is on the curve: ", (pk.y**2 - pk.x**3 - 7) % bitcoin_curve.p == 0)
# if it was 2, the public key is G + G
sk = 2
pk = G + G
print(f" secret key: {sk}\n public key: {(pk.x, pk.y)}")
print("Verify the public key is on the curve: ", (pk.y**2 - pk.x**3 - 7) % bitcoin_curve.p == 0)
# and so on...
sk = 3 
pk = G + G + G
print(f" secret key: {sk}\n public key: {(pk.x, pk.y)}")
print("Verify the public key is on the curve: ", (pk.y**2 - pk.x**3 - 7) % bitcoin_curve.p == 0)

 secret key: 1
 public key: (55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)
Verify the public key is on the curve:  True
 secret key: 2
 public key: (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930)
Verify the public key is on the curve:  True
 secret key: 3
 public key: (112711660439710606056748659173929673102114977341539408544630613555209775888121, 25583027980570883691656905877401976406448868254816295069919888960541586679410)
Verify the public key is on the curve:  True


### Alright so we have some keypairs above, but we want the public key associated with our randomly generated secret key above.  We have to add g to itself a large number of times, to make a large integer. It would work, but it runs slowly with what we have. Lets implement a "double and add" algorithm to dramatically speed this up:

In [8]:
def double_and_add(self, k: int) -> Point:
    assert isinstance(k, int) and k >= 0
    result = INF
    append = self
    while k:
        if k & 1:
            result += append
        append += append
        k >>= 1
    return result

# patch double and add into Point class
Point.__rmul__ = double_and_add

# verify that it work
print(G == 1*G)
print(G + G == 2*G)
print(G + G+ G == 3*G)

True
True
True


In [9]:
# Now lets try our actual public key
public_key = secret_key * G
print(f"x: {public_key.x}\ny: {public_key.y}")
print("Verify the public key is on the curve: ", (public_key.y**2 - public_key.x**3 - 7) % bitcoin_curve.p == 0)

x: 48748905458303824039212572068426481711982623479048540557208644708965566027762
y: 20208082702706679730357122151621917164185553236471908588011495134640180267134
Verify the public key is on the curve:  True


### Now that we have a public and private key, we have our crypto identity. Now its time to derive the associated Bitcoin wallet address. The wallet address is not just the public key, but it can be deterministically derived from it and has a few extra thing (such as embedded checksum).

### Before we can generate the address, we need some hash functions. Bitcoin uses SHA-256 and also RIPEMD-160. But we are doing this with no dependencies, so import hashlib is not allowed! Here is SHA-256  in pure Python:

In [10]:
def gen_sha256_variable_scope():
    """
    SHA256 Implementation.
    
    Follows the FIPS PUB 180-4 description for calculating SHA-256 hash functions at:
    https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
    
    This is for educational purposes, do not use for anything serious.
    """
    
    import math
    from itertools import count, islice
    
# -------------------------------------------------------------------------------------
# Shat-256 Functions, defined in section 4
    
    def rotr(x, n, size=32):
        return (x >> n) | (x << size - n) & (2**size - 1)
    
    def shr(x, n):
        return x >> n
    
    def sig0(x):
        return rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3)
    
    def sig1(x):
        return rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10)
    
    def capsig0(x):
        return rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22)
    
    def capsig1(x):
        return rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25)
    
    def ch(x, y, z):
        return (x & y)^ (~x & z)
    
    def maj(x, y, z):
        return (x & y) ^ (x & z) ^ (y & z)
    
    def b2i(b):
        return int.from_bytes(b, 'big')
    
    def i2b(i):
        return i.to_bytes(4, 'big')
    
# -------------------------------------------------------------------------------------
# SHAT-256 Constant

    def is_prime(n):
        return not any(f for f in range(2, int(math.sqrt(n))+1) if n%f == 0)

    def first_n_primes(n):
        return islice(filter(is_prime, count(start=2)), n)

    def frac_bin(f, n=32):
        """ Return the first n bits of fractional part of float f """
        f -= math.floor(f) # get only fractional part
        f *= 2**n # shift left
        f = int(f) # truncate teh rest of the fractional content
        return f

    def genK():
        """
        Follows Section 4.2.2 to generate K

        The first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers:

        428a2f98 71374491 b5c0fbcf e9b5dba5 3956c25b 59f111f1 923f82a4 ab1c5ed5
        d807aa98 12835b01 243185be 550c7dc3 72be5d74 80deb1fe 9bdc06a7 c19bf174
        e49b69c1 efbe4786 0fc19dc6 240ca1cc 2de92c6f 4a7484aa 5cb0a9dc 76f988da
        983e5152 a831c66d b00327c8 bf597fc7 c6e00bf3 d5a79147 06ca6351 14292967
        27b70a85 2e1b2138 4d2c6dfc 53380d13 650a7354 766a0abb 81c2c92e 92722c85
        a2bfe8a1 a81a664b c24b8b70 c76c51a3 d192e819 d6990624 f40e3585 106aa070
        19a4c116 1e376c08 2748774c 34b0bcb5 391c0cb3 4ed8aa4a 5b9cca4f 682e6ff3
        748f82ee 78a5636f 84c87814 8cc70208 90befffa a4506ceb bef9a3f7 c67178f2
        """
        return [frac_bin(p ** (1/3.0)) for p in first_n_primes(64)]

    def genH():
        """
        Follows Section 5.3.3 to generate the inital hash value H^0

        The first 32 bits of the fractional parts of the square roots of the first 8 prime numbers:

        6a09e667 bb67ae85 3c6ef372 a54ff53a 9b05688c 510e527f 1f83d9ab 5be0cd19
        """
        return [frac_bin(p ** (1/2.0)) for p in first_n_primes(8)]

    # -------------------------------------------------------------------------------------

    def pad(b):
        """ Follows Section 5.1: Padding the message """

        b = bytearray(b) # convert to a mutable equivalent
        l = len(b) * 8 # note: len returns number of bytes not bits

        # append but "1" to the end of the message
        b.append(0b10000000) # appending 10000000 in binary (=128 in decimal)

        # follow by k zero bits, where k is the smallest non-negative solution to
        # l + l + k = 448 mod 512
        # ie pad with zeros until we reach 448 (mod 512)

        while (len(b) * 8) % 512 != 448:
            b.append(0x00)

        # the last 64 bit block is the length l of the original message
        # expressed in binary (big endian)
        b.extend(l.to_bytes(8, 'big'))

        return b

    def sha256(b: bytes) -> bytes:
        # Section 4.2
        K = genK()

        # Section 5: Preprocessing
        # Section 5.1 Pad the message
        b = pad(b)
        # Section 5.2: Separate the message into blocks of 512 bits (64 bytes)
        blocks = [b[i: i+64] for i in range(0, len(b), 64)]

        # for each message block M^1 ... M^N
        H = genH() # Section 5.3

        # Section 6
        for M in blocks: # each block is a 64-entry arry of 8-bit bytes

            # 1. Prepare the message schedule, a 64-entry array of 8-bit bytes
            W = []
            for t in range(64):
                if t <= 15:
                    # the first 16 words are just a copy of the block
                    W.append(bytes(M[t*4:t*4+4]))
                else:
                    term1 = sig1(b2i(W[t-2]))
                    term2 = b2i(W[t-7])
                    term3 = sig0(b2i(W[t-15]))
                    term4 = b2i(W[t-16])
                    total = (term1 + term2 + term3 +term4) % 2**32
                    W.append(i2b(total))

            # 2. Initialize the 9 working variables a b c d e f g h with prev hash value
            a, b, c, d, e, f, g, h = H

            # 3.
            for t in range(64):
                T1 = (h + capsig1(e) + ch(e, f, g) + K[t] + b2i(W[t])) % 2**32
                T2 = (capsig0(a) + maj(a, b, c)) % 2**32
                h = g
                g = f
                f = e
                e = (d + T1) % 2**32
                d = c
                c = b
                b = a
                a = (T1 + T2) % 2**32

            # 4 Compute the i-th intermediate hash value H^i
            delta = [a, b, c, d, e, f, g, h]
            H = [(i1 + i2) % 2**32 for i1, i2 in zip(H, delta)]
        
        return b''.join(i2b(i) for i in H)
    
    return sha256

sha256 = gen_sha256_variable_scope()
print("verify empty hash:", sha256(b'').hex()) # should be e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
print(sha256(b'here is a random bytes message, cool right?').hex())
print("number of bytes in a sha256 digest: ", len(sha256(b'')))

verify empty hash: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
69b9779edaa573a509999cbae415d3408c30544bad09727a1d64eff353c95b89
number of bytes in a sha256 digest:  32


### Author notes that he stopped here to show that its nothing that scary: sha256 takes some bytes to be hashed, pads the message, breaks it up into chunks, then passes those chunks into what can be described as a "fancy mixer", with bitshifts and stuff.

### At the end you get a fixed-size, random looking short digest of any variable sized original message. The scrambling is not invertible and is thus-far impossible to construct a different message that hashes to the same digest.

### Bitcoin uses this everywhere, but its most interesting implementation is in mining: the ASICS basically used a very close to the metal form of the above code!

### We will also need the RIPEMD160 hash function, found on the internet and cleaned up

In [11]:
def gen_ripemd160_variable_scope():
    
    import sys
    import struct
    
    # -------------------------------------------------------------------------------------
    # public interface
    
    def ripemd160(b: bytes) -> bytes:
        """ Simple wraper for a simpler API to this hash function, just bytes to bytes """
        ctx = RMDContext()
        RMD160Update(ctx, b, len(b))
        digest = RMD160Final(ctx)
        return digest
    
    # -------------------------------------------------------------------------------------
    
    class RMDContext:
        def __init__(self):
            self.state = [0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476, 0xC3D2E1F0] # uint32
            self.count = 0 # uint64
            self.buffer = [0]*64 # uchar
            
    def RMD160Update(ctx, inp, inplen):
        have = int((ctx.count // 8) % 64)
        inplen = int(inplen)
        need = 64 - have
        ctx.count += 8 * inplen
        off = 0
        if inplen >= need:
            if have:
                for i in range(need):
                    ctx.buffer[have+i] = inp[i]
                RMD160Transform(ctx.state, ctx.buffer)
                off = need
                have = 0
            while off + 64 <= inplen:
                RMD160Transform(ctx.state, inp[off:])
                off += 64
        if off < inplen:
            for i in range(inplen - off):
                ctx.buffer[have+i] = inp[off+i]
                
    def RMD160Final(ctx):
        size = struct.pack("<Q", ctx.count)
        padlen = 64 - ((ctx.count // 8) % 64)
        if padlen < 1 + 8:
            padlen += 64
        RMD160Update(ctx, PADDING, padlen-8)
        RMD160Update(ctx, size, 8)
        return struct.pack("<5L", *ctx.state)
    
    # -------------------------------------------------------------------------------------
    
    
    K0 = 0x00000000
    K1 = 0x5A827999
    K2 = 0x6ED9EBA1
    K3 = 0x8F1BBCDC
    K4 = 0xA953FD4E
    KK0 = 0x50A28BE6
    KK1 = 0x5C4DD124
    KK2 = 0x6D703EF3
    KK3 = 0x7A6D76E9
    KK4 = 0x00000000
    
    PADDING = [0x80] + [0]*63
    
    def ROL(n, x):
        return ((x << n) & 0xffffffff) | (x >> (32 - n))
    
    def F0(x, y, z):
        return x ^ y ^ z

    def F1(x, y, z):
        return (x & y) | (((~x) % 0x100000000) & z)

    def F2(x, y, z):
        return (x | ((~y) % 0x100000000)) ^ z

    def F3(x, y, z):
        return (x & z) | (((~z) % 0x100000000) & y)

    def F4(x, y, z):
        return x ^ (y | ((~z) % 0x100000000))

    def R(a, b, c, d, e, Fj, Kj, sj, rj, X):
        a = ROL(sj, (a + Fj(b, c, d) + X[rj] + Kj) % 0x100000000) + e
        c = ROL(10, c)
        return a % 0x100000000, c
    
    def RMD160Transform(state, block): #uint32 state[5], uchar block[64]

        x = [0]*16
        assert sys.byteorder == 'little', "Only little endian is supported atm for RIPEMD160"
        x = struct.unpack('<16L', bytes(block[0:64]))

        a = state[0]
        b = state[1]
        c = state[2]
        d = state[3]
        e = state[4]

        #/* Round 1 */
        a, c = R(a, b, c, d, e, F0, K0, 11,  0, x)
        e, b = R(e, a, b, c, d, F0, K0, 14,  1, x)
        d, a = R(d, e, a, b, c, F0, K0, 15,  2, x)
        c, e = R(c, d, e, a, b, F0, K0, 12,  3, x)
        b, d = R(b, c, d, e, a, F0, K0,  5,  4, x)
        a, c = R(a, b, c, d, e, F0, K0,  8,  5, x)
        e, b = R(e, a, b, c, d, F0, K0,  7,  6, x)
        d, a = R(d, e, a, b, c, F0, K0,  9,  7, x)
        c, e = R(c, d, e, a, b, F0, K0, 11,  8, x)
        b, d = R(b, c, d, e, a, F0, K0, 13,  9, x)
        a, c = R(a, b, c, d, e, F0, K0, 14, 10, x)
        e, b = R(e, a, b, c, d, F0, K0, 15, 11, x)
        d, a = R(d, e, a, b, c, F0, K0,  6, 12, x)
        c, e = R(c, d, e, a, b, F0, K0,  7, 13, x)
        b, d = R(b, c, d, e, a, F0, K0,  9, 14, x)
        a, c = R(a, b, c, d, e, F0, K0,  8, 15, x) #/* #15 */
        #/* Round 2 */
        e, b = R(e, a, b, c, d, F1, K1,  7,  7, x)
        d, a = R(d, e, a, b, c, F1, K1,  6,  4, x)
        c, e = R(c, d, e, a, b, F1, K1,  8, 13, x)
        b, d = R(b, c, d, e, a, F1, K1, 13,  1, x)
        a, c = R(a, b, c, d, e, F1, K1, 11, 10, x)
        e, b = R(e, a, b, c, d, F1, K1,  9,  6, x)
        d, a = R(d, e, a, b, c, F1, K1,  7, 15, x)
        c, e = R(c, d, e, a, b, F1, K1, 15,  3, x)
        b, d = R(b, c, d, e, a, F1, K1,  7, 12, x)
        a, c = R(a, b, c, d, e, F1, K1, 12,  0, x)
        e, b = R(e, a, b, c, d, F1, K1, 15,  9, x)
        d, a = R(d, e, a, b, c, F1, K1,  9,  5, x)
        c, e = R(c, d, e, a, b, F1, K1, 11,  2, x)
        b, d = R(b, c, d, e, a, F1, K1,  7, 14, x)
        a, c = R(a, b, c, d, e, F1, K1, 13, 11, x)
        e, b = R(e, a, b, c, d, F1, K1, 12,  8, x) #/* #31 */
        #/* Round 3 */
        d, a = R(d, e, a, b, c, F2, K2, 11,  3, x)
        c, e = R(c, d, e, a, b, F2, K2, 13, 10, x)
        b, d = R(b, c, d, e, a, F2, K2,  6, 14, x)
        a, c = R(a, b, c, d, e, F2, K2,  7,  4, x)
        e, b = R(e, a, b, c, d, F2, K2, 14,  9, x)
        d, a = R(d, e, a, b, c, F2, K2,  9, 15, x)
        c, e = R(c, d, e, a, b, F2, K2, 13,  8, x)
        b, d = R(b, c, d, e, a, F2, K2, 15,  1, x)
        a, c = R(a, b, c, d, e, F2, K2, 14,  2, x)
        e, b = R(e, a, b, c, d, F2, K2,  8,  7, x)
        d, a = R(d, e, a, b, c, F2, K2, 13,  0, x)
        c, e = R(c, d, e, a, b, F2, K2,  6,  6, x)
        b, d = R(b, c, d, e, a, F2, K2,  5, 13, x)
        a, c = R(a, b, c, d, e, F2, K2, 12, 11, x)
        e, b = R(e, a, b, c, d, F2, K2,  7,  5, x)
        d, a = R(d, e, a, b, c, F2, K2,  5, 12, x) #/* #47 */
        #/* Round 4 */
        c, e = R(c, d, e, a, b, F3, K3, 11,  1, x)
        b, d = R(b, c, d, e, a, F3, K3, 12,  9, x)
        a, c = R(a, b, c, d, e, F3, K3, 14, 11, x)
        e, b = R(e, a, b, c, d, F3, K3, 15, 10, x)
        d, a = R(d, e, a, b, c, F3, K3, 14,  0, x)
        c, e = R(c, d, e, a, b, F3, K3, 15,  8, x)
        b, d = R(b, c, d, e, a, F3, K3,  9, 12, x)
        a, c = R(a, b, c, d, e, F3, K3,  8,  4, x)
        e, b = R(e, a, b, c, d, F3, K3,  9, 13, x)
        d, a = R(d, e, a, b, c, F3, K3, 14,  3, x)
        c, e = R(c, d, e, a, b, F3, K3,  5,  7, x)
        b, d = R(b, c, d, e, a, F3, K3,  6, 15, x)
        a, c = R(a, b, c, d, e, F3, K3,  8, 14, x)
        e, b = R(e, a, b, c, d, F3, K3,  6,  5, x)
        d, a = R(d, e, a, b, c, F3, K3,  5,  6, x)
        c, e = R(c, d, e, a, b, F3, K3, 12,  2, x) #/* #63 */
        #/* Round 5 */
        b, d = R(b, c, d, e, a, F4, K4,  9,  4, x)
        a, c = R(a, b, c, d, e, F4, K4, 15,  0, x)
        e, b = R(e, a, b, c, d, F4, K4,  5,  5, x)
        d, a = R(d, e, a, b, c, F4, K4, 11,  9, x)
        c, e = R(c, d, e, a, b, F4, K4,  6,  7, x)
        b, d = R(b, c, d, e, a, F4, K4,  8, 12, x)
        a, c = R(a, b, c, d, e, F4, K4, 13,  2, x)
        e, b = R(e, a, b, c, d, F4, K4, 12, 10, x)
        d, a = R(d, e, a, b, c, F4, K4,  5, 14, x)
        c, e = R(c, d, e, a, b, F4, K4, 12,  1, x)
        b, d = R(b, c, d, e, a, F4, K4, 13,  3, x)
        a, c = R(a, b, c, d, e, F4, K4, 14,  8, x)
        e, b = R(e, a, b, c, d, F4, K4, 11, 11, x)
        d, a = R(d, e, a, b, c, F4, K4,  8,  6, x)
        c, e = R(c, d, e, a, b, F4, K4,  5, 15, x)
        b, d = R(b, c, d, e, a, F4, K4,  6, 13, x) #/* #79 */

        aa = a
        bb = b
        cc = c
        dd = d
        ee = e

        a = state[0]
        b = state[1]
        c = state[2]
        d = state[3]
        e = state[4]

        #/* Parallel round 1 */
        a, c = R(a, b, c, d, e, F4, KK0,  8,  5, x)
        e, b = R(e, a, b, c, d, F4, KK0,  9, 14, x)
        d, a = R(d, e, a, b, c, F4, KK0,  9,  7, x)
        c, e = R(c, d, e, a, b, F4, KK0, 11,  0, x)
        b, d = R(b, c, d, e, a, F4, KK0, 13,  9, x)
        a, c = R(a, b, c, d, e, F4, KK0, 15,  2, x)
        e, b = R(e, a, b, c, d, F4, KK0, 15, 11, x)
        d, a = R(d, e, a, b, c, F4, KK0,  5,  4, x)
        c, e = R(c, d, e, a, b, F4, KK0,  7, 13, x)
        b, d = R(b, c, d, e, a, F4, KK0,  7,  6, x)
        a, c = R(a, b, c, d, e, F4, KK0,  8, 15, x)
        e, b = R(e, a, b, c, d, F4, KK0, 11,  8, x)
        d, a = R(d, e, a, b, c, F4, KK0, 14,  1, x)
        c, e = R(c, d, e, a, b, F4, KK0, 14, 10, x)
        b, d = R(b, c, d, e, a, F4, KK0, 12,  3, x)
        a, c = R(a, b, c, d, e, F4, KK0,  6, 12, x) #/* #15 */
        #/* Parallel round 2 */
        e, b = R(e, a, b, c, d, F3, KK1,  9,  6, x)
        d, a = R(d, e, a, b, c, F3, KK1, 13, 11, x)
        c, e = R(c, d, e, a, b, F3, KK1, 15,  3, x)
        b, d = R(b, c, d, e, a, F3, KK1,  7,  7, x)
        a, c = R(a, b, c, d, e, F3, KK1, 12,  0, x)
        e, b = R(e, a, b, c, d, F3, KK1,  8, 13, x)
        d, a = R(d, e, a, b, c, F3, KK1,  9,  5, x)
        c, e = R(c, d, e, a, b, F3, KK1, 11, 10, x)
        b, d = R(b, c, d, e, a, F3, KK1,  7, 14, x)
        a, c = R(a, b, c, d, e, F3, KK1,  7, 15, x)
        e, b = R(e, a, b, c, d, F3, KK1, 12,  8, x)
        d, a = R(d, e, a, b, c, F3, KK1,  7, 12, x)
        c, e = R(c, d, e, a, b, F3, KK1,  6,  4, x)
        b, d = R(b, c, d, e, a, F3, KK1, 15,  9, x)
        a, c = R(a, b, c, d, e, F3, KK1, 13,  1, x)
        e, b = R(e, a, b, c, d, F3, KK1, 11,  2, x) #/* #31 */
        #/* Parallel round 3 */
        d, a = R(d, e, a, b, c, F2, KK2,  9, 15, x)
        c, e = R(c, d, e, a, b, F2, KK2,  7,  5, x)
        b, d = R(b, c, d, e, a, F2, KK2, 15,  1, x)
        a, c = R(a, b, c, d, e, F2, KK2, 11,  3, x)
        e, b = R(e, a, b, c, d, F2, KK2,  8,  7, x)
        d, a = R(d, e, a, b, c, F2, KK2,  6, 14, x)
        c, e = R(c, d, e, a, b, F2, KK2,  6,  6, x)
        b, d = R(b, c, d, e, a, F2, KK2, 14,  9, x)
        a, c = R(a, b, c, d, e, F2, KK2, 12, 11, x)
        e, b = R(e, a, b, c, d, F2, KK2, 13,  8, x)
        d, a = R(d, e, a, b, c, F2, KK2,  5, 12, x)
        c, e = R(c, d, e, a, b, F2, KK2, 14,  2, x)
        b, d = R(b, c, d, e, a, F2, KK2, 13, 10, x)
        a, c = R(a, b, c, d, e, F2, KK2, 13,  0, x)
        e, b = R(e, a, b, c, d, F2, KK2,  7,  4, x)
        d, a = R(d, e, a, b, c, F2, KK2,  5, 13, x) #/* #47 */
        #/* Parallel round 4 */
        c, e = R(c, d, e, a, b, F1, KK3, 15,  8, x)
        b, d = R(b, c, d, e, a, F1, KK3,  5,  6, x)
        a, c = R(a, b, c, d, e, F1, KK3,  8,  4, x)
        e, b = R(e, a, b, c, d, F1, KK3, 11,  1, x)
        d, a = R(d, e, a, b, c, F1, KK3, 14,  3, x)
        c, e = R(c, d, e, a, b, F1, KK3, 14, 11, x)
        b, d = R(b, c, d, e, a, F1, KK3,  6, 15, x)
        a, c = R(a, b, c, d, e, F1, KK3, 14,  0, x)
        e, b = R(e, a, b, c, d, F1, KK3,  6,  5, x)
        d, a = R(d, e, a, b, c, F1, KK3,  9, 12, x)
        c, e = R(c, d, e, a, b, F1, KK3, 12,  2, x)
        b, d = R(b, c, d, e, a, F1, KK3,  9, 13, x)
        a, c = R(a, b, c, d, e, F1, KK3, 12,  9, x)
        e, b = R(e, a, b, c, d, F1, KK3,  5,  7, x)
        d, a = R(d, e, a, b, c, F1, KK3, 15, 10, x)
        c, e = R(c, d, e, a, b, F1, KK3,  8, 14, x) #/* #63 */
        #/* Parallel round 5 */
        b, d = R(b, c, d, e, a, F0, KK4,  8, 12, x)
        a, c = R(a, b, c, d, e, F0, KK4,  5, 15, x)
        e, b = R(e, a, b, c, d, F0, KK4, 12, 10, x)
        d, a = R(d, e, a, b, c, F0, KK4,  9,  4, x)
        c, e = R(c, d, e, a, b, F0, KK4, 12,  1, x)
        b, d = R(b, c, d, e, a, F0, KK4,  5,  5, x)
        a, c = R(a, b, c, d, e, F0, KK4, 14,  8, x)
        e, b = R(e, a, b, c, d, F0, KK4,  6,  7, x)
        d, a = R(d, e, a, b, c, F0, KK4,  8,  6, x)
        c, e = R(c, d, e, a, b, F0, KK4, 13,  2, x)
        b, d = R(b, c, d, e, a, F0, KK4,  6, 13, x)
        a, c = R(a, b, c, d, e, F0, KK4,  5, 14, x)
        e, b = R(e, a, b, c, d, F0, KK4, 15,  0, x)
        d, a = R(d, e, a, b, c, F0, KK4, 13,  3, x)
        c, e = R(c, d, e, a, b, F0, KK4, 11,  9, x)
        b, d = R(b, c, d, e, a, F0, KK4, 11, 11, x) #/* #79 */

        t = (state[1] + cc + d) % 0x100000000
        state[1] = (state[2] + dd + e) % 0x100000000
        state[2] = (state[3] + ee + a) % 0x100000000
        state[3] = (state[4] + aa + b) % 0x100000000
        state[4] = (state[0] + bb + c) % 0x100000000
        state[0] = t % 0x100000000

    return ripemd160

ripemd160 = gen_ripemd160_variable_scope()
print(ripemd160(b'hello this is a test').hex())
print("number of bytes in a RIPEMD-160 digest: ", len(ripemd160(b'')))

f51960af7dd4813a587ab26388ddab3b28d1f7b4
number of bytes in a RIPEMD-160 digest:  20


#### Author notes this is another bit scrambler.

#### Now to get a bitcoin address. We will create a subclass of Point called PublicKey which is just a point on the curve, but now it will have additional semantics and interpretation of a bitcoin public key, together with some encode/decode of the key into bytes for communication with the Bitcoin protocol

In [12]:
class PublicKey(Point):
    """
    The public key is just a Point on a Curve, but has some additional specific 
    encoding / decoding functionality that this class implements.
    """
    
    @classmethod
    def from_point(cls, pt: Point):
        """ promote a Point to be a PublicKey """
        return cls(pt.curve, pt.x, pt.y)
    
    def encode(self, compressed, hash160=False):
        """ Return the SEC bytes encoding of the public key Point """
        # calculate the bytes
        if compressed:
            # (x,y) is redundant, but due to the fact that this is modular arithmetic, there is 
            # no + or - in the terms, instead it can be shown that one y will always be even
            # and the other odd.
            prefix = b'\x02' if self.y % 2 == 0 else b'\x03'
            pkb = prefix + self.x.to_bytes(32, 'big')
        else:
            pkb = b'\x04' + self.x.to_bytes(32, 'big') + self.y.to_bytes(32, 'big')
            # hash if desired
        return ripemd160(sha256(pkb)) if hash160 else pkb
        
    def address(self, net: str, compressed: bool) -> str:
        """ Return the associated bitcoin address for this public key as string """
        # encode the public key into bytes and hash to get the payload
        pkb_hash = self.encode(compressed=compressed, hash160=True)
        # add version byte(0x00 for Main Network, or 0x6f for Test Network)
        version = {'main': b'\x00', 'test': b'\x6f'}
        ver_pkb_hash = version[net] + pkb_hash
        # calculate the checksum
        checksum = sha256(sha256(ver_pkb_hash))[:4]
        # append to form the full 25-byte binary Bitcoin Address
        byte_address = ver_pkb_hash + checksum
        # lastly, b58 encode teh result
        b58check_address = b58encode(byte_address)
        return b58check_address

#### Lastly, we need the b58encode function, which is Bitcoin specific and uses base 58 of characters in the alphabet that is very unambiguous ie it does not use O and 0, etc. The raw 25 bytes of the address contains 1 byte for version (to distinguish between mainnet and testnet bitcoin) and then 20 bytes of hash digest, and finally 4 bytes of a checksum so it can throw an error with very good probability in event of typo

In [13]:
# base58 encoding / decoding utilties
# reference https://en.bitcoin.it/wiki/Base58Check_encoding

alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'

def b58encode(b: bytes) -> str:
    assert len(b) == 25 # version is 1 byte, pkb_hash 20 bytes, checksum 4 bytes
    n = int.from_bytes(b, 'big')
    chars = []
    while n:
        n, i = divmod(n, 58)
        chars.append(alphabet[i])
        
    #special case handle the leading 0 bytes
    num_leading_zeros = len(b) - len(b.lstrip(b'\x00'))
    res = num_leading_zeros * alphabet[0] + ''.join(reversed(chars))
    return res

#### Now to print out a Bitcoin Address!

In [14]:
# This will use Testnet, so net='test'
address = PublicKey.from_point(public_key).address(net='test', compressed=True)
print(address)

mxZo3tmKf9qSre5QWCxsYzASTU9vhuLqvJ


#### Karpathy uses the address: mnNcaVkC35ezZSgvn8fhXEa9QTHSUtPfzQ     

#### You can use a block explorer to see it is a valid address on testnet that has been used: https://mempool.space/testnet/address/mnNcaVkC35ezZSgvn8fhXEa9QTHSUtPfzQ     

#### The Address I created has not been used yet: mxZo3tmKf9qSre5QWCxsYzASTU9vhuLqvJ

#### And this is what it looks like when the secret key is '1':

In [15]:
lol_secret_key = 1
lol_public_key = lol_secret_key * G
lol_address = PublicKey.from_point(lol_public_key).address(net='test', compressed=True)
lol_address

'mrCDrCybB6J1vRfbwM5hemdJz73FwDBC8r'

#### As of making this, this key has been used in 1851 transactions:
####  https://mempool.space/testnet/address/mrCDrCybB6J1vRfbwM5hemdJz73FwDBC8r
#### When Karpathy went through this, the number was 1812. It seems like people love using this address to test things! 

### Part 1 Summary
- We made an identity (secret key)
- Derived a public key using eliptic curve cryptography
- We then also derived the associated bitcoin address, by the introduction of two hash functions: SHA256 and RIPEMD160
- Here is a summary of the variables:

In [16]:
print("Our first Bitcoin identity:")
print("1. secret key: ", secret_key)
print("2. public key: ", (public_key.x, public_key.y))
print("3. Bitcoin address: ", address)

Our first Bitcoin identity:
1. secret key:  104966969588467315726631053946449615147380658266902276382595870948465674913455
2. public key:  (48748905458303824039212572068426481711982623479048540557208644708965566027762, 20208082702706679730357122151621917164185553236471908588011495134640180267134)
3. Bitcoin address:  mxZo3tmKf9qSre5QWCxsYzASTU9vhuLqvJ


#### Part 2 Obtaining funds and Bitcoin Under the Hood

It is now time to create a transaction. We will send some BTC from the above address (mxZo3tmKf9qSre5QWCxsYzASTU9vhuLqvJ) to a second wallet which we also control. Lets create that now:

In [17]:
secret_key2 = int.from_bytes(b"Algorant Super Secret 2nd Wallet", 'big') # or just random.randrange(1, bitcoin_gen.n)
assert 1 <= secret_key2 < bitcoin_gen.n # check it's valid
public_key2 = secret_key2 * G
address2 = PublicKey.from_point(public_key2).address(net='test', compressed=True)

print("Our second Bitcoin identity:")
print("1. secret key: ", secret_key2)
print("2. public key: ", (public_key2.x, public_key2.y))
print("3. Bitcoin address: ", address2)

Our second Bitcoin identity:
1. secret key:  29591868525381862461504049330911034654251218614528013635497042492899151603060
2. public key:  (19040609317612188280008065916145517715814950607708078129157603148751108655172, 53970884956107866087896829782642694344692727248560838622856592010825791549057)
3. Bitcoin address:  mzw1zXhrHyqoDyhKpgYtbBVV4T3cKoJss5


#### Alright now the goal would be to:
- Send Bitcoins to secret_key1 address(mxZo3tmKf9qSre5QWCxsYzASTU9vhuLqvJ) using a bitcoin testnet faucet
- send the bitcoins now on secret_key1 to secret_key2 address (mzw1zXhrHyqoDyhKpgYtbBVV4T3cKoJss5)

#### Using Testnet Faucet to get coins
I used coinfaucet.eu to send bitcoins to address1. They sent me 0.01813118BTC to that address. When I am finished, I will send them back as they request.

It was processed in Block 2064637, and the transaction ID was 238b924f2fb5a9d8eece5e2f7c08f56e196e2eb87ba78a6fa9b62300346ab35e

The fee was 14,801 sat, an overpay of 103x (haha). It used segwit and rbf opcodes.

#### Spending the coins
In order to spend the coins, a transaction must satisfy 2 conditions:
- the Public key needs to hash to match
- the digital signature for the next transaction must validate as being generated by the same private key

In [18]:
PublicKey.from_point(public_key).encode(compressed=True, hash160=True).hex()

'bb03ab69b18769898152474a7373ca8c8785f288'

### Part 3: Crafting the Transaction
We will now spend half the coins from the first wallet, and send them to the second wallet.  The transaction must be constructed in a way that the total amount of bitcoin currently associated with the first wallet is distributed exactly to two outputs: half to first wallet and half to second wallet. It must always be exact.

In [19]:
@dataclass
class TxIn:
    prev_tx: bytes # previous transaction ID: hash256 of previous tx contents
    prev_index: int # UTXO output index in the transaction
    script_sig: Script = None #Unlocking script, script class coming in below
    sequence: int = 0xffffffff #originall intented for "high frequency trades", with locktime
        
tx_in = TxIn(
    prev_tx = bytes.fromhex('238b924f2fb5a9d8eece5e2f7c08f56e196e2eb87ba78a6fa9b62300346ab35e'),
    prev_index = 1,
    script_sig = None, # this field will have digital signature, to be inserted later
)

#### Calculating the Fee
Now we will create a data structure for the two outputs of our transaction, instead of a market rate since this is testnet, we will just use arbitrary but very high fee (over 10sat/byte)

wallet1 = 1,813,118, or 1.8 million sat

The amounts have to be exact, so we will send 1.8 million sat, and return 1.0 million sat to us, leaving wallet 2 with 800,000 sat, and the remaining 13,118 satoshi will be the fee.

In [48]:
@dataclass
class TxOut:
    amount: int # in units of satoshi (1e-8 of a bitcoin)
    script_pubkey: Script = None # locking script

tx_out1 = TxOut(
    amount = 50000 # we will send this  sat to our target wallet
)
tx_out2 = TxOut(
    amount = 47000 # back to us
)
# the fee of 13,118 does not need to be manually specified, the miner will claim it

#### Populating the Locking Script
We will now populate the script_pubkey using native bitcoin scripting language. This is visible through any block explorer under the "Opcode" section. It will have a bunch of parameters by which the transaction was processed, all starting with "OP_". In this case it looks as follows:

OP_DUP
OP_HASH160
OP_PUSHBYTES_20 bb03ab69b18769898152474a7373ca8c8785f288
OP_EQUALVERIFY
OP_CHECKSIG

We will now create this same structure and encode it into bytes. We will also swap out the public key with the new owner's hashes. The op codes are all encoded as integers via a fixed schema.

In [49]:
def encode_int(i, nbytes, encoding='little'):
    """ encode integer i into nbytes using a given byte ordering """
    return i.to_bytes(nbytes, encoding)

def encode_varint(i):
    """ encode a (possibly but rarely large) integer into bytes with a super simple compression scheme """
    if i < 0xfd:
        return bytes([i])
    elif i < 0x10000:
        return b'\xfd' + encode_int(i, 2)
    elif i < 0x100000000:
        return b'\xfe' + encode_int(i, 4)
    elif i < 0x10000000000000000:
        return b'\xff' + encode_int(i, 8)
    else:
        raise ValueError("integer too large: %d" % (i, ))
        
@dataclass
class Script:
    cmds: List[Union[int, bytes]]
        
    def encode(self):
        out = []
        for cmd in self.cmds:
            if isinstance(cmd, int):
                # an int is just an opcode, encode as single byte
                out += [encode_int(cmd, 1)]
            elif isinstance(cmd, bytes):
                # bytes represent an element, encode its length and then content
                length = len(cmd)
                assert length < 75 # any longer than this requires a bit of tedious handling that we are skipping here
                
                out += [encode_int(length, 1), cmd]
                
        ret = b''.join(out)
        return encode_varint(len(ret)) + ret
    
# the first output will go to our 2nd wallet
out1_pkb_hash = PublicKey.from_point(public_key2).encode(compressed=True, hash160=True)
out1_script = Script([118, 169, out1_pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG
print(out1_script.encode().hex())

# the second output will go back to us
out2_pkb_hash = PublicKey.from_point(public_key).encode(compressed=True, hash160=True)
out2_script = Script([118, 169, out2_pkb_hash, 136, 172])
print(out2_script.encode().hex())                

1976a914d4f775b543860ea45e615168169c85bd4d98008588ac
1976a914bb03ab69b18769898152474a7373ca8c8785f28888ac


Now we are going to declare the owners of both inputs of our transaction by specifying the public key hashes (padded by these opcodes). We will see how these work shortly. For now, its just important t oknow that we are declaring the owner of each UTXO by identifying a specific public key hash. With the script above, only the person who has the original publc key(and its associated secret key) will be able to spend the UTXO.

In [50]:
tx_out1.script_pubkey = out1_script
tx_out2.script_pubkey = out2_script

#### Digital Signature

Now we are going to implement the signature that says "I, the owner of this private key associated with the public key in this transaction, approve the spending of this UTXO as the input of the transaction."

Bitcoin can get complex here as well, because you can only sign parts of transactions, but we will only be covering the most common and simple use case of signing and constructing the unlocking script (basically from above using the same opcodes).

We now run into the small problem of needing something that we don't yet have: we can't encode the transaction into bytes yet because it's still missing our signature, which we are still trying to construct.

This is confusing but it will hopefully be better with the code. We basically need the final structure of the signature, and then we can input the parts that we need.

In [51]:
@dataclass
class Tx:
    version: int
    tx_ins: List[TxIn]
    tx_outs: List[TxOut]
    locktime: int = 0
    
    def encode(self, sig_index=-1) -> bytes:
        """
        Encode this transaction as bytes.
        If sig_index is given then return the modified transaction
        encoding of this tx with respect to the single input index.
        This result then constitutes the "message" that gets signed
        by the aspiring transactor of this input.
        """
        out = []
        # encode metadata
        out += [encode_int(self.version, 4)]
        # encode inputs
        out += [encode_varint(len(self.tx_ins))]
        if sig_index == -1:
            # we are just serializing a fully formed transaction
            out += [tx_in.encode() for tx_in in self.tx_ins]
        else:
            # used when crafting digital signature for a specific input index
            out += [tx_in.encode(script_override=(sig_index == i))
                    for i, tx_in in enumerate(self.tx_ins)]
            # encode outputs
            out += [encode_varint(len(self.tx_outs))]
            out += [tx_out.encode() for tx_out in self.tx_outs]
            # encode... other metadata
            out += [encode_int(self.locktime, 4)]
            out += [encode_int(1, 4) if sig_index != -1 else b''] # 1 = SIGHASH_ALL
            return b''.join(out)
        
# we also need to know how to encode TxIn. This is just serialization protocol.
def txin_encode(self, script_override=None):
    out = []
    out += [self.prev_tx[::-1]] # little endian vs big endian encodings... sigh
    out += [encode_int(self.prev_index, 4)]
    
    if script_override is None:
        # None = just use the actual script
        out += [self.script_sig.encode()]
    elif script_override is True:
        # True = override the script with the script_pubkey of the associated input
        out += [self.prev_tx_script_pubkey.encode()]
    elif script_override is False:
        # False = override with an empty script
        out += [Script([]).encode()]
    else:
        raise ValueError("script_override must be one of None|True|False")
        
    out += [encode_int(self.sequence, 4)]
    return b''.join(out)

TxIn.encode = txin_encode # monkey patch into the class

# and TxOut as well
def txout_encode(self):
    out = []
    out += [encode_int(self.amount, 8)]
    out += [self.script_pubkey.encode()]
    return b''.join(out)

TxOut.encode = txout_encode # monkey patch into the class

tx = Tx(
    version = 1,
    tx_ins = [tx_in],
    tx_outs = [tx_out1, tx_out2],
)

We now need to get the PUSHBYTES_20 opcode output back in raw_bytes form. We will re-create it as a script:

In [52]:
source_script = Script([118, 169, out2_pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG
print("recall out2_pkb_hash is just raw bytes of hash of public key: ", out2_pkb_hash.hex())
print(source_script.encode().hex()) # we can get the bytes of the script_pubkey now

recall out2_pkb_hash is just raw bytes of hash of public key:  bb03ab69b18769898152474a7373ca8c8785f288
1976a914bb03ab69b18769898152474a7373ca8c8785f28888ac


In [53]:
# monkey patch this into the input of the transaction we are trying to sign and construct
tx_in.prev_tx_script_pubkey = source_script

# get the "message" we need to digitally sign!!
message = tx.encode(sig_index = 0)
message.hex()

'01000000015eb36a340023b6a96f8aa77bb82e6e196ef5087c2f5eceeed8a9b52f4f928b23010000001976a914bb03ab69b18769898152474a7373ca8c8785f28888acffffffff0250c30000000000001976a914d4f775b543860ea45e615168169c85bd4d98008588ac98b70000000000001976a914bb03ab69b18769898152474a7373ca8c8785f28888ac0000000001000000'

This message basically encodes: the inputs, the new outputs (new UTXOs), their amounts, and their owners (via the locking scripts specifying the public key hash of each owner)

We are now ready to digitally sign the message with our private key. It is a tuple of two integers (r, s) using ECDSA.  Here is the code:

In [54]:
@dataclass
class Signature:
    r: int
    s: int

def sign(secret_key: int, message: bytes) -> Signature:
    
    # the order of the elliptic curve used in bitcoin
    n = bitcoin_gen.n
    
    # double hash the message and convert to integer
    z = int.from_bytes(sha256(sha256(message)), 'big')
    
    # generate a new secret/public key pair at random
    sk = random.randrange(1, n)
    P = sk * bitcoin_gen.G
    
    # calculate the signature
    r = P.x
    s = inv(sk, n) * (z + secret_key * r) % n
    if s > n / 2:
        s = n - s
        
    sig = Signature(r, s)
    return sig

def verify(public_key: Point, message: bytes, sig: Signature) -> bool:
    # just a stub reference on how a signature would be verified in terms of an API
    # we don't need to verify to craft a transaction, but we would if we were mining
    pass

random.seed(int.from_bytes(sha256(message), 'big')) # see note below
sig = sign(secret_key, message)
sig

Signature(r=9353957952797451715798014068878488502701403054618457780900952214086507718511, s=44596542932761857528776859008070212761491781800175751772982601224193991724942)

This is the naive form of generating a random number. Please do not use this anywhere near anything that touches production. There are specific standards for generating sk deterministically. It is skipped here for brevity and instead it is seeded rng with a hash of the message.

My signature: Signature(r=111150740185293687862399167138014157389525385266632941462944024201815267813370, s=5195644674013439821437534413262359366364787640626146377183955062223134734962)

Now we will implement the ```encode``` function of ```Signature``` so that we can broadcast it over the Bitcoin protocol. To do so we are using [DER Encoding](https://en.bitcoin.it/wiki/BIP_0062#DER_encoding)

In [55]:
def signature_encode(self) -> bytes:
    """ return the DER encoding of this signature """
    
    def dern(n):
        nb = n.to_bytes(32, byteorder='big')
        nb = nb.lstrip(b'\x00') # strip leading zeros
        nb = (b'\x00' if nb[0] >= 0x80 else b'') + nb # prepend 0x00 if first byte >= 0x80
        return nb
    
    rb = dern(self.r)
    sb = dern(self.s)
    content = b''.join([bytes([0x02, len(rb)]), rb, bytes([0x02, len(sb)]), sb])
    frame = b''.join([bytes([0x30, len(content)]), content])
    return frame

Signature.encode = signature_encode #monkey patch into class
sig_bytes = sig.encode()
sig_bytes.hex()


'3044022014ae270fb7c6df05fe3ddc4398592dcf05328599a016c217309e2d596c7a676f02206298bfb174b9c0a1bfba80d4d561d85f1be510e3fef8ad574eb2ae876c574f8e'

We are now finally ready to generate the ```script sig``` from the single input of our transaction. It will contain exactly two elements: the signature, and the public key, both encoded as bytes:

In [56]:
# Append 1 (=SIGHASH_ALL), indicating this DER signature encoded "ALL" of the tx (by far most common)

sig_bytes_and_type = sig_bytes + b'\x01'

# Encode the public key into bytes, we use hash160=False so it is the full public key to the Blockchain
pubkey_bytes = PublicKey.from_point(public_key).encode(compressed=True, hash160=False)

# Create a lightweight Script that just encodes these two things
script_sig = Script([sig_bytes_and_type, pubkey_bytes])
tx_in.script_sig = script_sig

So now we have created both locking scripts ```(script_pubkey)``` and the unlocking script ```(script_sig)``` and now we can talk about how they interact. On a high level, in each transaction input the two strips get concatenated into a single script, which then runs through the bitcoin code. The scripts will look like this:
```
<sig_bytes_and_type>
<pubkey_bytes>
OP_DUP
OP_HASH160
<pubkey_hash_bytes>
OP_EQUALVERIFY
OP_CHECKSIG
```

Then this gets executed with a typical stack-based push/pop scheme, where any bytes get pushed into the stack, and any ops will consume some inputs and push some outputs. So the pubkey gets duplicated (OP_DUP) and then gets hashed (OP_HASH160), then the hash gets compared to ```pubkey_hash_bytes``` (OP_EQUALVERIFY), and finally the digital signature integrity is verified as having been signed by the associated private key. 

We have now completed all the necessary steps. Lets look at the fully constructed transaction again:


In [57]:
tx

Tx(version=1, tx_ins=[TxIn(prev_tx=b'#\x8b\x92O/\xb5\xa9\xd8\xee\xce^/|\x08\xf5n\x19n.\xb8{\xa7\x8ao\xa9\xb6#\x004j\xb3^', prev_index=1, script_sig=Script(cmds=[b"0D\x02 \x14\xae'\x0f\xb7\xc6\xdf\x05\xfe=\xdcC\x98Y-\xcf\x052\x85\x99\xa0\x16\xc2\x170\x9e-Ylzgo\x02 b\x98\xbf\xb1t\xb9\xc0\xa1\xbf\xba\x80\xd4\xd5a\xd8_\x1b\xe5\x10\xe3\xfe\xf8\xadWN\xb2\xae\x87lWO\x8e\x01", b"\x02k\xc6\xe7\x17\xa6\xf6\xad\xea\xc9\x15\x90\xe2\x8fM\xe1\xebM{ _\xe5\x93\xd1\xe4'+\x11\x0f\xabr\x97\xf2"]), sequence=4294967295)], tx_outs=[TxOut(amount=50000, script_pubkey=Script(cmds=[118, 169, b'\xd4\xf7u\xb5C\x86\x0e\xa4^aQh\x16\x9c\x85\xbdM\x98\x00\x85', 136, 172])), TxOut(amount=47000, script_pubkey=Script(cmds=[118, 169, b'\xbb\x03\xabi\xb1\x87i\x89\x81RGJss\xca\x8c\x87\x85\xf2\x88', 136, 172]))], locktime=0)

Now lets encode it into bytes and show it in hex:

In [58]:
tx.encode().hex()

AttributeError: 'NoneType' object has no attribute 'hex'

In [45]:
print("Transaction size in bytes: ", len(tx.encode()))

TypeError: object of type 'NoneType' has no len()

In [46]:
print(tx.encode())

None


In [47]:
def tx_id(self) -> str:
    return sha256(sha256(self.encode()))[::-1].hex() # little/big endian conventions require byte order swap
Tx.id = tx_id # monkey patch into the class

tx.id() # once this transaction goes through, this will be its id

TypeError: cannot convert 'NoneType' object to bytearray